## oct 8 new plan

Use this!! https://github.com/damian0815/finetune-clip-huggingface/blob/main/huggingface_finetune_clip_runner.ipynb

In [ ]:
!pip install pandas requests

In [ ]:
import requests
import pandas as pd

url = 'https://raw.githubusercontent.com/opencontext/archaeology-images-ai/main/json_data/artifact_images_w_descriptions.json'
data = requests.get(url).json()
df = pd.json_normalize(data)  # convert json to pandas DataFrame

In [ ]:
df.rename(columns={'image_file__uri': 'image'}, inplace=True)
df

,image,media__uri,image_genre,image_type,subject__item_class__label,context___1,context___2,context___3,time_range,has_type,consists_of,origin_place,has_taxonomic_identifier,has_anatomical_identification,temporal_coverage,project_specific_descriptions
0,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/a9cedbad-e25b-4f...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,seals (artifacts),rock (inorganic material),None,None,None,None,"Artifact Name: Stamp Seal \n Material: Stone, ..."
1,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/1bbbca07-82f3-46...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,seals (artifacts),soapstone,None,None,None,None,Artifact Name: Stamp Seal \n Material: Steatit...
2,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/2062e3fa-41e2-d7...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,seals (artifacts),rock (inorganic material),None,None,None,None,"Artifact Name: Stamp Seal \n Material: Stone, ..."
3,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/2dc18114-4ddf-7c...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,pendants (jewelry),chert,None,None,None,None,Artifact Name: Pendant \n Material: Chert/Flint
4,https://iiif.archivelab.org/iiif/opencontext-1...,https://opencontext.org/media/d7e8b4e5-be3b-44...,archaeology,artifact,Object,Asia,Turkey,Domuztepe,6500 BCE to 5500 BCE,nails (fasteners),iron (metal),None,None,None,None,Artifact Name: Nail \n Material: Iron \n Dispo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72357,https://artiraq.org/static/opencontext/pettegr...,https://opencontext.org/media/d3620d27-cb41-44...,archaeology,artifact,Object,Europe,Greece,Corinthia,,None,None,None,None,None,None,"Chronotype: Fineware, Late Helladic I-IIA \n Z..."
72358,https://artiraq.org/static/opencontext/pettegr...,https://opencontext.org/media/41f03708-baa5-4d...,archaeology,artifact,Object,Europe,Greece,Corinthia,,None,None,None,None,None,None,"Chronotype: Fineware, Late Helladic I-IIA \n Z..."
72359,https://artiraq.org/static/opencontext/pettegr...,https://opencontext.org/media/453e04b2-7905-4e...,archaeology,artifact,Object,Europe,Greece,Corinthia,,None,None,None,None,None,None,"Chronotype: Fineware, Late Helladic I-IIA \n Z..."
72360,https://artiraq.org/static/opencontext/interna...,https://opencontext.org/media/1840719d-2934-48...,archaeology,artifact,Object,Off World,International Space Station,Zvezda Service Module,,icons (devotional images); religions and reli...,None,None,None,None,None,Location: Top center \n Item type: Icon \n Sec...


In [ ]:
import os
import random
import requests

download_dir = 'images' # directory where you want to store images
random_images = df.sample(500, random_state=67)  # pick 500 random rows

os.makedirs(download_dir, exist_ok=True)
url_errors=[]
for _, row in random_images.iterrows():
    url = row['image']
    # Get the extension for the image file from the URL
    extension = url.split('.')[-1]
    # Get the UUID for the media file
    media_uuid = row['media__uri'].split('/')[-1]

    # Make a unique file_name from the UUID of the Open Context media resource. This has
    # the advantage of making sure that the image files can be easily looked up on
    # Open Context itself.
    file_name = f'{media_uuid}.{extension}'
    file_path = os.path.join('images', file_name)

    try:
        response = requests.get(url, timeout=5)
        response.raise_for_status()
    except (requests.exceptions.RequestException, requests.exceptions.Timeout):
        print(f'An error occurred while fetching: {url}')
        url_errors.append(url)
        continue

    with open(file_path, 'wb') as img_file:
        img_file.write(response.content)

An error occurred while fetching: https://iiif.archivelab.org/iiif/opencontext-22-f-7-7005-7312-1-p-3jpg/full/675,/0/default.jpg
An error occurred while fetching: https://artiraq.org/static/opencontext/poggio-civitate/preview/photos/20030129PROFILE.jpg
An error occurred while fetching: https://iiif.archivelab.org/iiif/opencontext-22-d-6-188-16-1-p-4jpg/full/675,/0/default.jpg
An error occurred while fetching: https://iiif.archivelab.org/iiif/opencontext-24-19900011exteriorbtjpg/full/675,/0/default.jpg


In [ ]:
import string
import re

def create_metadata(df):
    # fill NaN values with empty string
    df.fillna('', inplace=True)

    # append values of required columns into new 'caption' column
    df['caption'] = 'A photograph of ' + df['consists_of'].astype(str) + ', ' + df['project_specific_descriptions'].astype(str) + ' dating to ' + df['time_range'].astype(str) + ' from ' + df['context___1'].astype(str) + ', ' + df['context___2'].astype(str) + ', ' + df['context___3'].astype(str)
    df['caption'] = df['caption'].replace('\n', ' ')
    # Remove all other punctuation.
    df['caption'] = df['caption'].apply(lambda x: re.sub(r'[{}]'.format(string.punctuation), ' ', x)).str.strip()

    # Rewrite 'image' column to just contain the filename (extract 'media__uuid' from the url and add the extension)
    df['image'] = df.apply(lambda row: f"{row['media__uri'].split('/')[-1]}.{row['image'].split('.')[-1]}", axis=1)

    # reshaping data to contain only 'image' and 'caption'
    df = df[['image', 'caption']]
    df['image'] = 'images/' + df['image'].astype(str)

    return df

df = create_metadata(random_images) #the output.json should have the same number of records as files in images
with open('output.json', 'w') as file:
    df.to_json(file, orient='records', lines=True)

<ipython-input-153-aa9005f8778b>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['image'] = 'images/' + df['image'].astype(str)


In [ ]:
!pip install torchvision datasets Pillow
!pip install -q git+https://github.com/huggingface/transformers
!pip install accelerate -U

In [ ]:
# test loading it back in
from datasets import load_dataset
dataset = load_dataset("json", data_files="output.json")
print(f"first image: {dataset['train'][0]['image']}, caption: '{dataset['train'][0]['caption']}'")


first image: a8ea6c5f-bdc5-45ca-9d41-9023a9152c1e.jpg, caption: 'A photograph of pottery  Fragment Noted  false 
 Fabric Category  Impasto 
 Object Type  Textile Related  Spindle Whorl 
 Decorative Technique  Stamped dating to 700 BCE to 535 BCE from Europe  Italy  Poggio Civitate'


In [ ]:
!git clone https://github.com/damian0815/finetune-clip-huggingface.git

Cloning into 'finetune-clip-huggingface'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 19 (delta 6), reused 17 (delta 4), pack-reused 0
Receiving objects: 100% (19/19), 13.79 KiB | 4.60 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
!mkdir results

In [ ]:
repo_id =  "openai/clip-vit-base-patch32"
output_folder = "results"
batch_size = 20
num_train_epochs = 60
out_json = "output.json"

In [168]:
print(f"Finetuning {repo_id} for {num_train_epochs} epochs with batch size {batch_size}, and then saving output to {output_folder}.")
!python -W ignore finetune-clip-huggingface/huggingface_finetune_clip.py \
    --output_dir {output_folder} \
    --model_name_or_path {repo_id} \
    --train_file {out_json} \
    --image_column image \
    --overwrite_output_dir=True \
    --max_seq_length=77 \
    --num_train_epochs={num_train_epochs} \
    --caption_column caption \
    --remove_unused_columns=False \
    --do_train \
    --per_device_train_batch_size={batch_size} \
    --learning_rate="5e-5" --warmup_steps="2" --weight_decay 0.2
print("--\nDONE")
print(f"If it worked, trained data should be in {output_folder}")

Finetuning openai/clip-vit-base-patch32 for 30 epochs with batch size 20, and then saving output to results.
2023-10-08 18:29:32.105289: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
10/08/2023 18:29:33 - WARNING - __main__ - Process rank: 0, device: cuda:0, n_gpu: 1distributed training: True, 16-bits training: False
Parameter 'transform'=<function main.<locals>.transform_images at 0x7a0aa935a7a0> of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
10/08/2023 18:29:36 - WARNING - datasets.fingerprint - Parameter 'transform'=<function

In [169]:
!zip -r archaeai.zip results/pytorch_model.bin results/config.json

updating: results/pytorch_model.bin (deflated 7%)
updating: results/config.json (deflated 46%)


In [170]:
from google.colab import files
files.download("archaeai.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>